In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.4 MB/s eta 0:00:00


In [ ]:
import sys
import random
import torch
import torch.nn as nn
import shutil
import numpy as np
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import time
import datetime
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

In [ ]:
if torch.cuda.is_available():
    # torch.cuda.set_device(0)
    device = torch.device('cuda')
    print('Using GPU: ', torch.cuda.current_device())
else:
    device = torch.device('cpu')

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(device)

Using GPU:  0
cuda


In [ ]:
import pandas as pd
import csv
import string

In [ ]:
df = pd.read_csv('/content/Bhaav-Dataset.csv',sep=',')
df

,Sentences,Annotation
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1
4,गाँव में कितनी हलचल है,1
...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1


In [ ]:
df1 = df.assign(joy=0)
df2 = df1.assign(anger=0)
df3 = df2.assign(suspense=0)
df4 = df3.assign(sad=0)
train_df = df4.assign(neutral=0)

In [ ]:
train_df

,Sentences,Annotation,joy,anger,suspense,sad,neutral
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1,0,0,0,0,0
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1,0,0,0,0,0
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1,0,0,0,0,0
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1,0,0,0,0,0
4,गाँव में कितनी हलचल है,1,0,0,0,0,0
...,...,...,...,...,...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4,0,0,0,0,0
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4,0,0,0,0,0
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4,0,0,0,0,0
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1,0,0,0,0,0


In [ ]:
for index in train_df.index:
    k = train_df['Annotation'][index]
    if k == 0:
        train_df['joy'][index] = 0
        train_df['anger'][index] = 1
        train_df['suspense'][index] = 0
        train_df['sad'][index] = 0
        train_df['neutral'][index] = 0
    elif k == 1:
        train_df['joy'][index] = 1
        train_df['anger'][index] = 0
        train_df['suspense'][index] = 0
        train_df['sad'][index] = 0
        train_df['neutral'][index] = 0
    elif k == 2:
        train_df['joy'][index] = 0
        train_df['anger'][index] = 0
        train_df['suspense'][index] = 0
        train_df['sad'][index] = 1
        train_df['neutral'][index] = 0
    elif k == 3:
        train_df['joy'][index] = 0
        train_df['anger'][index] = 0
        train_df['suspense'][index] = 1
        train_df['sad'][index] = 0
        train_df['neutral'][index] = 0
    elif k==4:
        train_df['joy'][index] = 0
        train_df['anger'][index] = 0
        train_df['suspense'][index] = 0
        train_df['sad'][index] = 0
        train_df['neutral'][index] = 1
train_df

<ipython-input-8-cee5a35a340a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['joy'][index] = 1
<ipython-input-8-cee5a35a340a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['anger'][index] = 0
<ipython-input-8-cee5a35a340a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['suspense'][index] = 0
<ipython-input-8-cee5a35a340a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,Sentences,Annotation,joy,anger,suspense,sad,neutral
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1,1,0,0,0,0
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1,1,0,0,0,0
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1,1,0,0,0,0
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1,1,0,0,0,0
4,गाँव में कितनी हलचल है,1,1,0,0,0,0
...,...,...,...,...,...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4,0,0,0,0,1
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4,0,0,0,0,1
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4,0,0,0,0,1
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1,1,0,0,0,0


In [ ]:
categories = train_df.columns[2:]
counts = []
for category in categories:
    counts.append((category, train_df[category].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number of comments'])
df_stats

,category,number of comments
0,joy,2463
1,anger,1464
2,suspense,1512
3,sad,3168
4,neutral,11697


In [ ]:
target_list = categories

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

In [ ]:
def tokenizeWithBert(example):
  encodings = tokenizer.encode_plus(
    example,
    add_special_tokens = True,   # tokens CLS, PAD, SEP
    max_length = 512, #MAX_LEN
    padding = 'max_length',
    truncation = True,
    return_attention_mask = True,
    return_tensors = 'pt'
  )
  return encodings

In [ ]:
def get_dataset(df, tokenizer, mode='train'):
    sentences, labels = df['Sentences'], df.iloc[:,2:].to_numpy()
    max_length = 300
    in_T = []
    in_T_attn_masks = []
    for sentence in sentences:
        enc_sent_dict = tokenizer.encode_plus(
            sentence[:300],
            max_length = max_length,
            add_special_tokens = True,
            pad_to_max_length = True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )
        in_T.append(enc_sent_dict['input_ids'])
        in_T_attn_masks.append(enc_sent_dict['attention_mask'])
    
    in_T = torch.cat(in_T, dim=0)
    in_T_attn_masks = torch.cat(in_T_attn_masks, dim=0)
    labels = torch.tensor(labels, dtype = torch.float32)
    print('Text Input: ' , in_T.shape)
    print('Text Input Attention: ' , in_T_attn_masks.shape)    
    print('Labels: ' , labels.shape)
    
    dataset = TensorDataset(
        in_T,
        in_T_attn_masks,
        labels
    )
    
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_dataset, val_dataset = get_dataset(
    train_df,
    tokenizer = tokenizer,
    mode = 'train'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Text Input:  torch.Size([20304, 300])
Text Input Attention:  torch.Size([20304, 300])
Labels:  torch.Size([20304, 5])


In [ ]:
batch_size = 8
train_dataloader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    sampler = RandomSampler(train_dataset)
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size = batch_size,
    sampler = SequentialSampler(val_dataset)
)

print('Data Ready!!')

Data Ready!!


In [ ]:
import torch.nn.functional as F
from transformers import BertModel

class MultiClassClassifier(nn.Module):
    def __init__(self, hidden_dim, num_labels):
        super(MultiClassClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_labels = num_labels
        
        self.bertmodel = BertModel.from_pretrained('bert-base-multilingual-uncased')
        self.ffn1 = nn.Linear(768, hidden_dim)
        self.dp1 = nn.Dropout()
        self.ffn2 = nn.Linear(hidden_dim, num_labels)
        
    def forward(self, in_T, in_T_attn_masks):
        outputs = self.bertmodel(in_T, in_T_attn_masks)
        #from 3.x onwards use outputs.last_hidden_state
        x = torch.mean(outputs.last_hidden_state, dim=1) #from 3.x onwards use outputs.last_hidden_state
        x = F.relu(self.ffn1(x))
        x = self.dp1(x)
        x = torch.sigmoid(self.ffn2(x))
        return x

In [ ]:
#format time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
model = MultiClassClassifier(100, 5).to(device) # 100 hidden dimension, 
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8) # Adam with weight decay
criterion = nn.BCELoss()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#TRAINING and VALIDATION
epochs = 3   #5, reduced to one epoch as it is taking lot of time
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = total_steps)


training_stats = []
total_t0 = time.time()

best_val_loss = 1e8
true_labels = val_dataset[:][2].numpy()

for epoch_i in range(0, epochs):

    #############               Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 5 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}. Loss: {:.5f}'.format(step, len(train_dataloader), elapsed, total_train_loss/step))

        b_in_T            = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)
        b_labels          = batch[2].to(device)
        
        model.zero_grad()

        logits = model(b_in_T, b_in_T_attn_masks)
        loss = criterion(logits, b_labels)

        total_train_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)

   
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

   
   
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    pred_labels = np.empty((0,5))

    # Evaluate data for one epoch
    for batch in val_dataloader:
        
        b_in_T            = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)
        b_labels          = batch[2].to(device)

        with torch.no_grad():
            logits = model(b_in_T, b_in_T_attn_masks)
            loss = criterion(logits, b_labels)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        pred_labels = np.concatenate((pred_labels, logits), axis=0)


    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(val_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    pred_labels = np.array([[int(x >= 0.25) for x in pred_labels[:,i]] for i  in range(5)]).transpose()

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

#     Report the final accuracy, f1-score for this validation run.
    for i in range(5):
        print("  Accuracy: {0:.2f}".format(accuracy_score(true_labels[:,i], pred_labels[:,i])))

    for i in range(5):
        print("  Macro F1-score: {0:.2f}".format(f1_score(true_labels[:,i], pred_labels[:,i], average='macro')))

    for i in range(5):
        print("  Weighted F1-score: {0:.2f}".format(f1_score(true_labels[:,i], pred_labels[:,i], average='weighted')))

    print('Classification Report:')
    for i in range(5):
        print(classification_report(true_labels[:,i], pred_labels[:,i]))

    print('Confusion Matrix:')
    for i in range(5):
        print(confusion_matrix(true_labels[:,i], pred_labels[:,i]))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'training_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'val_accuracy': np.mean([accuracy_score(true_labels[:,i], pred_labels[:,i]) for i in range(5)]),
            'val_macro_f1': np.mean([f1_score(true_labels[:,i], pred_labels[:,i], average='macro') for i in range(5)]),
            'val_weighted_f1': np.mean([f1_score(true_labels[:,i], pred_labels[:,i], average='weighted') for i in range(5)]),
            'training_time': training_time,
            'val_tim': validation_time
        }
    )

    model_path = 'model_state_dict_'+str(epoch_i)+'.pt'
    torch.save(model.state_dict(), model_path)


======== Epoch 1 / 3 ========
Training...
  Batch     5  of  2,031.    Elapsed: 0:00:02. Loss: 0.38588
  Batch    10  of  2,031.    Elapsed: 0:00:05. Loss: 0.44436
  Batch    15  of  2,031.    Elapsed: 0:00:07. Loss: 0.41155
  Batch    20  of  2,031.    Elapsed: 0:00:10. Loss: 0.39802
  Batch    25  of  2,031.    Elapsed: 0:00:12. Loss: 0.41028
  Batch    30  of  2,031.    Elapsed: 0:00:14. Loss: 0.40921
  Batch    35  of  2,031.    Elapsed: 0:00:17. Loss: 0.40455
  Batch    40  of  2,031.    Elapsed: 0:00:19. Loss: 0.41126
  Batch    45  of  2,031.    Elapsed: 0:00:22. Loss: 0.40916
  Batch    50  of  2,031.    Elapsed: 0:00:24. Loss: 0.40393
  Batch    55  of  2,031.    Elapsed: 0:00:27. Loss: 0.40008
  Batch    60  of  2,031.    Elapsed: 0:00:29. Loss: 0.39405
  Batch    65  of  2,031.    Elapsed: 0:00:32. Loss: 0.39192
  Batch    70  of  2,031.    Elapsed: 0:00:35. Loss: 0.39193
  Batch    75  of  2,031.    Elapsed: 0:00:37. Loss: 0.39011
  Batch    80  of  2,031.    Elapsed: 0:00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



======== Epoch 2 / 3 ========
Training...
  Batch     5  of  2,031.    Elapsed: 0:00:02. Loss: 0.44713
  Batch    10  of  2,031.    Elapsed: 0:00:05. Loss: 0.41154
  Batch    15  of  2,031.    Elapsed: 0:00:07. Loss: 0.39968
  Batch    20  of  2,031.    Elapsed: 0:00:10. Loss: 0.40975
  Batch    25  of  2,031.    Elapsed: 0:00:12. Loss: 0.40388
  Batch    30  of  2,031.    Elapsed: 0:00:14. Loss: 0.40521
  Batch    35  of  2,031.    Elapsed: 0:00:17. Loss: 0.41216
  Batch    40  of  2,031.    Elapsed: 0:00:19. Loss: 0.41806
  Batch    45  of  2,031.    Elapsed: 0:00:22. Loss: 0.41833
  Batch    50  of  2,031.    Elapsed: 0:00:24. Loss: 0.42241
  Batch    55  of  2,031.    Elapsed: 0:00:26. Loss: 0.42505
  Batch    60  of  2,031.    Elapsed: 0:00:29. Loss: 0.42018
  Batch    65  of  2,031.    Elapsed: 0:00:31. Loss: 0.42285
  Batch    70  of  2,031.    Elapsed: 0:00:34. Loss: 0.42093
  Batch    75  of  2,031.    Elapsed: 0:00:36. Loss: 0.42195
  Batch    80  of  2,031.    Elapsed: 0:00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


======== Epoch 3 / 3 ========
Training...
  Batch     5  of  2,031.    Elapsed: 0:00:02. Loss: 0.44313
  Batch    10  of  2,031.    Elapsed: 0:00:05. Loss: 0.42626
  Batch    15  of  2,031.    Elapsed: 0:00:07. Loss: 0.44258
  Batch    20  of  2,031.    Elapsed: 0:00:09. Loss: 0.43377
  Batch    25  of  2,031.    Elapsed: 0:00:12. Loss: 0.43501
  Batch    30  of  2,031.    Elapsed: 0:00:14. Loss: 0.43308
  Batch    35  of  2,031.    Elapsed: 0:00:17. Loss: 0.42686
  Batch    40  of  2,031.    Elapsed: 0:00:19. Loss: 0.43117
  Batch    45  of  2,031.    Elapsed: 0:00:22. Loss: 0.42931
  Batch    50  of  2,031.    Elapsed: 0:00:24. Loss: 0.42756
  Batch    55  of  2,031.    Elapsed: 0:00:26. Loss: 0.42588
  Batch    60  of  2,031.    Elapsed: 0:00:29. Loss: 0.42719
  Batch    65  of  2,031.    Elapsed: 0:00:31. Loss: 0.42543
  Batch    70  of  2,031.    Elapsed: 0:00:34. Loss: 0.42140
  Batch    75  of  2,031.    Elapsed: 0:00:36. Loss: 0.42419
  Batch    80  of  2,031.    Elapsed: 0:00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif